# Statistics from HAWC2

If you have run many HAWC2 simulations, perhaps on a cluster or using a batch script, it is useful to visualize the statistics. The two steps are to first post-process and then visualize the results.

## Process time series into stats file

Once time-series files have been generated with HAWC2, you can calculate statistics from all of the simulations and save the results into an CSV file. Making this statistics file is part of the LAC course and is therefore intentionally not explained here. The stats file can be with or without the 10-minute damage-equivalent loads (DELs). Calculating without includes base statistics such as mean, standard deviation, max, min, etc.

## Load stats file and examine its structure

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from lacbox.io import load_stats
from lacbox.test import test_data_path

stats_path = Path(test_data_path) / 'dtu_10mw_steady_stats.hdf5'
h2stats, wsps = load_stats(stats_path, statstype='steady')
type(h2stats)

The `h2stats` object is essentially a `pandas.DataFrame` but with a few additions. Let's first familiarize ourselves with this object and the structure of the stats file.

In [ ]:
print('The stats file is located here:', h2stats.statspath)
print('The shape is:', h2stats.shape)
print('The column names are:', h2stats.columns)

There are 9040 rows in this stats file, each corresponding to a single channel in a single file. The columns are:  
 * `path`: the full path to the HAWC2 time-marching result file  
 * `filename`: the name of the HAWC2 file  
 * `subfolder`: the name of the subfolder, if the htc file was in one  
 * `ichan`: the channel index, matching pdap (i.e., starting from 1)  
 * `names`: the name of the channel, matching pdap  
 * `units`: the units of the channel, matching pdap  
 * `desc`: the description of the channel, matching pdap  
 * `mean`, `max`, etc.: the value of the corresponding statistic  
 * `X%`: the value of the corresponding percentile  
 * `delX`: the 10-minute damage-equivalent fatigue load, calculated with the indicated value for the Wöhler exponent  

This dataset is the DTU 10 MW operating with steady wind, no tower shadow, and no shear. The dataset includes four sets of simulations: with normal tilt, with no tilt, with no tilt and rigid tower/blades, and no tilt with rigid tower/blades and no aerodynamic drag. Each case was saved in a subfolder, and we can examine their names.

In [ ]:
h2stats.subfolder.unique()

## Isolate channels of interest and make plots

Let's make a plot of some data versus mean wind speed for the two cases to investigate the effects of tilt on mean loads.

We want to be able to isolate channel based on a human-friendly identifier. The `filter_channel()` method in `HAWC2Stats` object can do that. But we need to define a dictionary that maps channel identifiers to substrings expected in that channel's description. Here is that dictionary for this model:

In [4]:
CHAN_DESCS = {'BldPit': 'pitch1 angle',  # blade pitch angle
              'RotSpd': 'rotor speed',  # rotor speed
              'Thrust': 'aero rotor thrust',  # thrust
              'GenTrq': 'generator torque',  # generator torque
              'ElPow': 'pelec',  # electrical poewr
              'TbFA': 'momentmx mbdy:tower nodenr:   1',  # tower base FA
              'TbSS': 'momentmy mbdy:tower nodenr:   1',  # tower base SS
              'FlpBRM': 'momentmx mbdy:blade1 nodenr:   1 coo: blade1',  # flapwise blade-root moment
              'EdgBRM': 'momentmy mbdy:blade1 nodenr:   1 coo: blade1',  # edgewise blade-root moment
              }

Now let's isolate two cases, perhaps with and without tilt.

In [5]:
tilt_stats = h2stats[h2stats.subfolder == 'tilt']
notilt_stats = h2stats[h2stats.subfolder == 'notilt']

And finally let's plot our channels of interest in a quick plot.

In [ ]:
plot_chans = ['RotSpd', 'BldPit', 'ElPow', 'GenTrq', 'TbFA', 'TbSS', 'FlpBRM', 'EdgBRM']
fig, axs = plt.subplots(4, 2, num=1, figsize=(12, 10), clear=True)
for i, chan_id in enumerate(plot_chans):
    ax = axs.flatten()[i]
    # get the mean value for that channel
    tilt_chan = tilt_stats.filter_channel(chan_id, CHAN_DESCS)
    notilt_chan = notilt_stats.filter_channel(chan_id, CHAN_DESCS)
    # plot versus mean wsp
    ax.plot(tilt_chan['wsp'], tilt_chan['mean'], 'o', label='With tilt')
    ax.plot(notilt_chan['wsp'], notilt_chan['mean'], 'x', label='Without tilt')
    # prettify
    ax.grid()
    ax.set(title=chan_id)

axs[0, 0].legend()

fig.tight_layout()

We can see that tilt does not significantly impact the steady-state operational values for the turbine or the tower-base fore-aft moment. However, it does have an impact on the tower-base side-side moment and some impact on the blades as well.

Let's identify which simulation has the largest tower-base for-aft moment for both cases.

In [ ]:
chan_id = 'TbFA'

idx_max = tilt_stats.filter_channel(chan_id, CHAN_DESCS)['mean'].idxmax()
print('Max-TBFA simulation with tilt:', tilt_stats.loc[idx_max, 'filename'])
print('  Max load:', tilt_stats.loc[idx_max, 'mean'], 'kNm')

idx_max = notilt_stats.filter_channel(chan_id, CHAN_DESCS)['mean'].idxmax()
print('Max-TBFA simulation without tilt:', notilt_stats.loc[idx_max, 'filename'])
print('  Max load:', notilt_stats.loc[idx_max, 'mean'], 'kNm')

As expected, both have peaks at 11 m/s, which is near rated.